In [49]:
import pandas as pd
import os
import string
from textblob import TextBlob
import nltk
from nltk import word_tokenize
from nltk.util import ngrams
from collections import Counter

In [2]:
import spacy
nlp = spacy.load("fr_core_news_sm")

In [3]:
dates_GDN = ["31-01-19","06-02-19","17-02-19","02-03-19","08-03-19","21-03-19"]
sujets_GDN = {"ecologie": "LA_TRANSITION_ECOLOGIQUE",
             "democratie": "DEMOCRATIE_ET_CITOYENNETE",
             "fisc": "LA_FISCALITE_ET_LES_DEPENSES_PUBLIQUES",
             "etat": "ORGANISATION_DE_LETAT_ET_DES_SERVICES_PUBLICS"}
sujets_VD = os.listdir("data/VD")

In [4]:
sujets_VD

['2019-03-04_justice-police-armee_consultation-3.csv',
 '2019-03-06_expression-libre_consultation-9.csv',
 '2019-03-04_democratie-institutions-referendum-dinitiative-citoyenne_consultation.csv',
 '2019-03-06_transition-ecologique-solidaire-agriculture-alimentation_consultation-2.csv',
 '2019-03-04_europe-affaires-etrangeres-outre-mer_consultation-4.csv',
 '2019-03-04_sport-culture_consultation-8.csv',
 '.~lock.2019-03-04_justice-police-armee_consultation-3.csv#',
 '2019-03-04_education-jeunesse-enseignement-superieur-recherche-et-innovation_consultation-7.csv',
 '2019-03-04_economie-finances-travail-compte-public_consultation-6.csv',
 '2019-03-04_sante-solidarite-handicap_consultation-5.csv']

In [44]:
flatten = lambda l: [item for sublist in l for item in sublist]

### Fichier utilisé actuellement pour les tests:

`2019-03-04_justice-police-armee_consultation-3.csv`

In [5]:
df = pd.read_csv(f"data/VD/{sujets_VD[0]}")

/usr/lib/python3/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (7,11,12,13,34,35,36,42,43,46,47,50,51,54,55,56,57,59,66) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [36]:
df

,type,contributions_id,contributions_author_id,contributions_section_title,contributions_title,contributions_bodyText,contributions_createdAt,contributions_updatedAt,contributions_url,contributions_published,...,contributions_sources_body,contributions_sources_createdAt,contributions_sources_updatedAt,contributions_sources_published,contributions_sources_votesCount,contribution_versions_id,contribution_versions_title,contribution_versions_bodyText,contribution_versions_createdAt,contribution_versions_updatedAt
0,opinion,2a6dce90-3c3a-11e9-988a-fa163e5b3bfd,VXNlcjo3MzIwNTEwZi0zYzBlLTExZTktOTg4YS1mYTE2M2...,"Justice, Police, Armée",Fin de la dissuasion nucléaire,Mettre plutôt nos investissements dans les out...,2019-03-01 16:53:32,NaN,https://le-vrai-debat.fr/consultations/justice...,Yes,...,NaN,2019-03-01 16:53:32,NaN,Yes,1.0,2a6dce90-3c3a-11e9-988a-fa163e5b3bfd,Fin de la dissuasion nucléaire,Mettre plutôt nos investissements dans les out...,2019-03-01 16:53:32,NaN
1,vote,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,opinion,d2a3afbb-3c39-11e9-988a-fa163e5b3bfd,VXNlcjo3MzIwNTEwZi0zYzBlLTExZTktOTg4YS1mYTE2M2...,"Justice, Police, Armée",Retour des forces militaires françaises engagé...,Mettre fin à une politique militaire d'ingéren...,2019-03-01 16:51:05,NaN,https://le-vrai-debat.fr/consultations/justice...,Yes,...,NaN,2019-03-01 16:51:05,NaN,Yes,1.0,d2a3afbb-3c39-11e9-988a-fa163e5b3bfd,Retour des forces militaires françaises engagé...,Mettre fin à une politique militaire d'ingéren...,2019-03-01 16:51:05,NaN
3,vote,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,opinion,bf49a981-3c1a-11e9-988a-fa163e5b3bfd,VXNlcjpkNzcwOTFjYi0yOWZlLTExZTktOTg4YS1mYTE2M2...,"Justice, Police, Armée",Gestion citoyenne radars fixes,Création de groupes de citoyens non profession...,2019-03-01 13:08:38,NaN,https://le-vrai-debat.fr/consultations/justice...,Yes,...,NaN,2019-03-01 13:08:38,NaN,Yes,1.0,bf49a981-3c1a-11e9-988a-fa163e5b3bfd,Gestion citoyenne radars fixes,Création de groupes de citoyens non profession...,2019-03-01 13:08:38,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33002,vote,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33003,vote,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33004,vote,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33005,vote,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
body = df[["contributions_bodyText"]].dropna()
body.reset_index(drop=True, inplace=True)

In [11]:
body.shape[0]

1400

-> 1400 Propositions au total

In [7]:
#Les propositions tokenisées
body["tokens"] = body["contributions_bodyText"].apply(lambda doc: [mot for mot in nlp(doc) if not mot.is_stop and not mot.is_punct])

In [13]:
#Nombre de tokens par proposition
body["propLen"] = body["tokens"].apply(lambda l: len(l))

In [16]:
#Nombre de phrases par proposition
body["nbPhrases"] = body["contributions_bodyText"].apply(lambda doc: len(TextBlob(doc).sentences))

In [23]:
#unigram
body["unigram"] = body["tokens"].apply(lambda tokens: list(ngrams(tokens,1)))

In [32]:
#bigram
body["bigram"] = body["tokens"].apply(lambda tokens: list(ngrams(tokens,2)))

In [33]:
body

,contributions_bodyText,tokens,propLen,nbPhrases,unigram,bigram
0,Mettre plutôt nos investissements dans les out...,"[Mettre, investissements, outils, cyberguerre,...",9,1,"[(Mettre,), (investissements,), (outils,), (cy...","[(Mettre, investissements), (investissements, ..."
1,Mettre fin à une politique militaire d'ingéren...,"[Mettre, fin, politique, militaire, ingérence,...",6,1,"[(Mettre,), (fin,), (politique,), (militaire,)...","[(Mettre, fin), (fin, politique), (politique, ..."
2,Création de groupes de citoyens non profession...,"[Création, groupes, citoyens, professionnels, ...",38,4,"[(Création,), (groupes,), (citoyens,), (profes...","[(Création, groupes), (groupes, citoyens), (ci..."
3,Que les votes blancs soient pris en compte.Plu...,"[votes, blancs, soient, pris, compte, temps, p...",23,3,"[(votes,), (blancs,), (soient,), (pris,), (com...","[(votes, blancs), (blancs, soient), (soient, p..."
4,Revaloriser les salaires de nos citoyens trava...,"[Revaloriser, salaires, citoyens, travaillant,...",19,2,"[(Revaloriser,), (salaires,), (citoyens,), (tr...","[(Revaloriser, salaires), (salaires, citoyens)..."
...,...,...,...,...,...,...
1395,Il serait utile qu'une police municipale exist...,"[utile, police, municipale, existe, communauté...",17,1,"[(utile,), (police,), (municipale,), (existe,)...","[(utile, police), (police, municipale), (munic..."
1396,Trafic de drogue : Saisie des biens du trafiqu...,"[Trafic, drogue, , Saisie, biens, trafiquant,...",8,1,"[(Trafic,), (drogue,), ( ,), (Saisie,), (biens...","[(Trafic, drogue), (drogue, ), ( , Saisie), (..."
1397,"En ce qui concerne la justice, il est psycholo...","[concerne, justice, psychologiquement, nécessa...",63,6,"[(concerne,), (justice,), (psychologiquement,)...","[(concerne, justice), (justice, psychologiquem..."
1398,"Bonjour à tous, Citoyens, à l'heure actuelle, ...","[Bonjour, Citoyens, heure, actuelle, magistrat...",90,3,"[(Bonjour,), (Citoyens,), (heure,), (actuelle,...","[(Bonjour, Citoyens), (Citoyens, heure), (heur..."


In [34]:
body["bigram"][0]

[(Mettre, investissements),
 (investissements, outils),
 (outils, cyberguerre),
 (cyberguerre, neutralisation),
 (neutralisation, distance),
 (distance, infrastructures),
 (infrastructures, électriques),
 (électriques, etc.)]

In [10]:
body["tokens"][0]

[Mettre,
 investissements,
 outils,
 cyberguerre,
 neutralisation,
 distance,
 infrastructures,
 électriques,
 etc.]

In [54]:
unigram = flatten(body["unigram"].tolist())
bigrams = flatten(body["bigram"].tolist())

In [55]:
bigramCounter = Counter(bigrams)
unigramCounter = Counter(unigram)

In [56]:
unigramCounter

Counter({(Mettre,): 1,
         (investissements,): 1,
         (outils,): 1,
         (cyberguerre,): 1,
         (neutralisation,): 1,
         (distance,): 1,
         (infrastructures,): 1,
         (électriques,): 1,
         (etc.,): 1,
         (Mettre,): 1,
         (fin,): 1,
         (politique,): 1,
         (militaire,): 1,
         (ingérence,): 1,
         (Afrique,): 1,
         (Création,): 1,
         (groupes,): 1,
         (citoyens,): 1,
         (professionnels,): 1,
         (chargés,): 1,
         (vérifier,): 1,
         (radars,): 1,
         (fixes,): 1,
         (soient,): 1,
         (installés,): 1,
         (sites,): 1,
         (accidentogènes,): 1,
         (groupes,): 1,
         (possibilité,): 1,
         (faire,): 1,
         (désactiver,): 1,
         (radars,): 1,
         (abusifs,): 1,
         (chargés,): 1,
         (faire,): 1,
         (installer,): 1,
         (panneaux,): 1,
         (avertisseurs,): 1,
         (indications,): 1,
         

In [5]:
print(os.listdir("data/VD"))

['2019-03-04_justice-police-armee_consultation-3.csv', '2019-03-06_expression-libre_consultation-9.csv', '2019-03-04_democratie-institutions-referendum-dinitiative-citoyenne_consultation.csv', '2019-03-06_transition-ecologique-solidaire-agriculture-alimentation_consultation-2.csv', '2019-03-04_europe-affaires-etrangeres-outre-mer_consultation-4.csv', '2019-03-04_sport-culture_consultation-8.csv', '2019-03-04_education-jeunesse-enseignement-superieur-recherche-et-innovation_consultation-7.csv', '2019-03-04_economie-finances-travail-compte-public_consultation-6.csv', '2019-03-04_sante-solidarite-handicap_consultation-5.csv']


In [149]:
df = pd.read_csv(f"data/GDN/{sujets_GDN['ecologie']}.csv")

/usr/lib/python3/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,52,56,57,59) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [150]:
df

,Unnamed: 0,"Avez-vous pour vos déplacements quotidiens la possibilité de recourir à des solutions de mobilité alternatives à la voiture individuelle comme les transports en commun, le covoiturage, l'auto-partage, le transport à la demande, le vélo, etc. ?",Diriez-vous que votre vie quotidienne est aujourd'hui touchée par le changement climatique ?,Et qui doit selon vous se charger de vous proposer ce type de solutions alternatives ?,"Par rapport à votre mode de chauffage actuel, pensez-vous qu'il existe des solutions alternatives plus écologiques ?",Q146 - Diriez-vous que votre vie quotidienne est aujourd'hui touchée par le changement climatique ?,"Q147 - Si oui, de quelle manière votre vie quotidienne est-elle touchée par le changement climatique ?","Q148 - À titre personnel, pensez-vous pouvoir contribuer à protéger l'environnement ?","Q149 - Si oui, que faites-vous aujourd'hui pour protéger l'environnement et/ou que pourriez-vous faire ?","Q150 - Qu'est-ce qui pourrait vous inciter à changer vos comportements comme par exemple mieux entretenir et régler votre chauffage, modifier votre manière de conduire ou renoncer à prendre votre véhicule pour de très petites distances ?",Q151 - Quelles seraient pour vous les solutions les plus simples et les plus supportables sur un plan financier pour vous inciter à changer vos comportements ?,"Q152 - Par rapport à votre mode de chauffage actuel, pensez-vous qu'il existe des solutions alternatives plus écologiques ?","Q153 - Si oui, que faudrait-il faire pour vous convaincre ou vous aider à changer de mode de chauffage ?","Q154 - Avez-vous pour vos déplacements quotidiens la possibilité de recourir à des solutions de mobilité alternatives à la voiture individuelle comme les transports en commun, le covoiturage, l'auto-partage, le transport à la demande, le vélo, etc. ?","Q155 - Si oui, que faudrait-il faire pour vous convaincre ou vous aider à utiliser ces solutions alternatives ?",Q157 - Et qui doit selon vous se charger de vous proposer ce type de solutions alternatives ?,Q158 - Que pourrait faire la France pour faire partager ses choix en matière d'environnement au niveau européen et international ?,Q159 - Y a-t-il d'autres points sur la transition écologique sur lesquels vous souhaiteriez vous exprimer ?,Q160 - Quel est aujourd'hui pour vous le problème concret le plus important dans le domaine de l'environnement ?,Q161 - Que faudrait-il faire selon vous pour apporter des réponses à ce problème ?,"Q207 - Si non, quelles sont les solutions de mobilité alternatives que vous souhaiteriez pouvoir utiliser ?",QUXVlc3Rpb246MTQ2 - Diriez-vous que votre vie quotidienne est aujourd'hui touchée par le changement climatique ?,"QUXVlc3Rpb246MTQ3 - Si oui, de quelle manière votre vie quotidienne est-elle touchée par le changement climatique ?","QUXVlc3Rpb246MTQ4 - À titre personnel, pensez-vous pouvoir contribuer à protéger l'environnement ?","QUXVlc3Rpb246MTQ5 - Si oui, que faites-vous aujourd'hui pour protéger l'environnement et/ou que pourriez-vous faire ?","QUXVlc3Rpb246MTU0 - Avez-vous pour vos déplacements quotidiens la possibilité de recourir à des solutions de mobilité alternatives à la voiture individuelle comme les transports en commun, le covoiturage, l'auto-partage, le transport à la demande, le vélo, etc. ?","QUXVlc3Rpb246MTU1 - Si oui, que faudrait-il faire pour vous convaincre ou vous aider à utiliser ces solutions alternatives ?",QUXVlc3Rpb246MTU3 - Et qui doit selon vous se charger de vous proposer ce type de solutions alternatives ?,QUXVlc3Rpb246MTU4 - Que pourrait faire la France pour faire partager ses choix en matière d'environnement au niveau européen et international ?,QUXVlc3Rpb246MTU5 - Y a-t-il d'autres points sur la transition écologique sur lesquels vous souhaiteriez vous exprimer ?,"QUXVlc3Rpb246MTUw - Qu'est-ce qui pourrait vous inciter à changer vos comportements comme par exemple mieux entretenir et régler votre chauffage, modifier votre manière de conduire ou re

In [147]:
ids = list(set(hd for hd in df.columns if "id" in hd))
ids

['contributions_sources_related_id',
 'contributions_reportings_related_id',
 'contributions_reportings_id',
 'contributions_votes_author_id',
 'contributions_votes_id',
 'contributions_arguments_id',
 'contributions_id',
 'contribution_versions_id',
 'contributions_arguments_related_id',
 'contributions_arguments_author_id',
 'contributions_reportings_author_id',
 'contributions_sources_id',
 'contributions_author_id',
 'contributions_sources_author_id']

In [145]:
test_id = "bf354656-2d2b-11e9-988a-fa163e5b3bfd"
ids.pop(0)

df.loc[df[ids[0]]==test_id]

IndexError: list index out of range

In [114]:
df[df.contributions_votesCount > 10]

,type,contributions_id,contributions_author_id,contributions_section_title,contributions_title,contributions_bodyText,contributions_createdAt,contributions_updatedAt,contributions_url,contributions_published,contributions_trashed,contributions_trashedStatus,contributions_trashedAt,contributions_trashedReason,contributions_votesCount,contributions_votesCountOk,contributions_votesCountMitige,contributions_votesCountNok,contributions_argumentsCount,contributions_argumentsCountFor,contributions_argumentsCountAgainst,contributions_sourcesCount,contributions_versionsCount,contributions_arguments_related_id,contributions_arguments_related_kind,contributions_arguments_id,contributions_arguments_author_id,contributions_arguments_type,contributions_arguments_body,contributions_arguments_createdAt,contributions_arguments_updatedAt,contributions_arguments_url,contributions_arguments_published,contributions_arguments_trashed,contributions_arguments_trashedStatus,contributions_arguments_trashedAt,contributions_arguments_trashedReason,contributions_arguments_votesCount,contributions_votes_id,contributions_votes_author_id,contributions_votes_value,contributions_votes_createdAt,contributions_reportings_related_id,contributions_reportings_related_kind,contributions_reportings_id,contributions_reportings_author_id,contributions_reportings_type,contributions_reportings_body,contributions_reportings_createdAt,contributions_sources_id,contributions_sources_related_id,contributions_sources_related_kind,contributions_sources_author_id,contributions_sources_trashed,contributions_sources_trashedStatus,contributions_sources_trashedAt,contributions_sources_trashedReason,contributions_sources_body,contributions_sources_createdAt,contributions_sources_updatedAt,contributions_sources_published,contributions_sources_votesCount,contribution_versions_id,contribution_versions_title,contribution_versions_bodyText,contribution_versions_createdAt,contribution_versions_updatedAt
984,opinion,bf354656-2d2b-11e9-988a-fa163e5b3bfd,VXNlcjo5NDUzMjE4Yy0yYzhkLTExZTktOTg4YS1mYTE2M2...,Sport / Culture,Stopper la défiscalisation de l'art contemporain,Supprimer la défiscalisation de l'art contempo...,2019-02-10 13:02:32,NaN,https://le-vrai-debat.fr/consultations/sport-c...,Yes,No,NaN,NaN,NaN,16.0,15.0,0.0,1.0,3.0,2.0,1.0,0.0,0.0,NaN,NaN,bf354656-2d2b-11e9-988a-fa163e5b3bfd,VXNlcjo5NDUzMjE4Yy0yYzhkLTExZTktOTg4YS1mYTE2M2...,NaN,NaN,2019-02-10 13:02:32,NaN,https://le-vrai-debat.fr/consultations/sport-c...,Yes,No,NaN,NaN,NaN,16.0,bf354656-2d2b-11e9-988a-fa163e5b3bfd,VXNlcjo5NDUzMjE4Yy0yYzhkLTExZTktOTg4YS1mYTE2M2...,NaN,2019-02-10 13:02:32,NaN,NaN,bf354656-2d2b-11e9-988a-fa163e5b3bfd,VXNlcjo5NDUzMjE4Yy0yYzhkLTExZTktOTg4YS1mYTE2M2...,NaN,NaN,2019-02-10 13:02:32,bf354656-2d2b-11e9-988a-fa163e5b3bfd,NaN,NaN,VXNlcjo5NDUzMjE4Yy0yYzhkLTExZTktOTg4YS1mYTE2M2...,No,NaN,NaN,NaN,NaN,2019-02-10 13:02:32,NaN,Yes,16.0,bf354656-2d2b-11e9-988a-fa163e5b3bfd,Stopper la défiscalisation de l'art contemporain,Supprimer la défiscalisation de l'art contempo...,2019-02-10 13:02:32,NaN
1014,opinion,c55921b4-2d29-11e9-988a-fa163e5b3bfd,VXNlcjplMDYwZGQxMS0yOTRmLTExZTktYmU4OS0wMjQyYW...,Sport / Culture,Arrêt de la pub sur les chaînes de TV publiques,Interdire totalement la publicité sur la télév...,2019-02-10 12:48:23,NaN,https://le-vrai-debat.fr/consultations/sport-c...,Yes,No,NaN,NaN,NaN,12.0,12.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,c55921b4-2d29-11e9-988a-fa163e5b3bfd,VXNlcjplMDYwZGQxMS0yOTRmLTExZTktYmU4OS0wMjQyYW...,NaN,NaN,2019-02-10 12:48:23,NaN,https://le-vrai-debat.fr/consultations/sport-c...,Yes,No,NaN,NaN,NaN,12.0,c55921b4-2d29-11e9-988a-fa163e5b3bfd,VXNlcjplMDYwZGQxMS0yOTRmLTExZTktYmU4OS0wMjQyYW...,NaN,2019-02-10 12:48:23,NaN,NaN,c55921b4-2d29-11e9-988a-fa163e5b3bfd,VXNlcjplMDYwZGQxMS0yOTRmLTExZTktYmU4OS0wMjQyYW...,NaN,NaN,2019-02-10 12:48:23,c55921b4-2d29-11e9-988a-fa163e5b3bfd,NaN,NaN,VXNlcjplMDYwZGQxMS0yOTRmLTExZTktYmU4OS0wMjQyYW...,No,NaN,NaN,NaN,NaN,2019-02-10 12:48:23,NaN,Yes,12.0,c55921b4-2d

In [36]:
data2 = data2.drop("contributions_votes_author_id", axis=1)

In [37]:
data2

,type,contributions_title,contributions_bodyText,contributions_createdAt,contributions_published,contributions_trashed,contributions_trashedStatus,contributions_trashedAt,contributions_trashedReason,contributions_votesCount,contributions_votesCountOk,contributions_votesCountMitige,contributions_votesCountNok,contributions_argumentsCount,contributions_argumentsCountFor,contributions_argumentsCountAgainst,contributions_sourcesCount,contributions_versionsCount,contributions_arguments_related_kind,contributions_arguments_type,contributions_arguments_body,contributions_arguments_createdAt,contributions_arguments_published,contributions_arguments_trashed,contributions_arguments_trashedStatus,contributions_arguments_trashedAt,contributions_arguments_trashedReason,contributions_arguments_votesCount,contributions_votes_value,contributions_votes_createdAt,contributions_reportings_related_id,contributions_reportings_related_kind,contributions_reportings_id,contributions_reportings_author_id,contributions_reportings_type,contributions_reportings_body,contributions_reportings_createdAt,contributions_sources_id,contributions_sources_related_id,contributions_sources_related_kind,contributions_sources_author_id,contributions_sources_trashed,contributions_sources_trashedStatus,contributions_sources_trashedAt,contributions_sources_trashedReason,contributions_sources_body,contributions_sources_createdAt,contributions_sources_updatedAt,contributions_sources_published,contributions_sources_votesCount,contribution_versions_id,contribution_versions_title,contribution_versions_bodyText,contribution_versions_createdAt,contribution_versions_updatedAt
0,opinion,Donner accès aux spectacles au plus grand nomb...,Développer la diffusion vidéo de spectacles un...,2019-02-28 13:50:35,Yes,No,NaN,NaN,NaN,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,2019-02-28 13:50:35,Yes,No,NaN,NaN,NaN,1.0,NaN,2019-02-28 13:50:35,NaN,NaN,713fd69b-3b57-11e9-988a-fa163e5b3bfd,VXNlcjpkZWNhNjI2Yi0zYjU1LTExZTktOTg4YS1mYTE2M2...,NaN,NaN,2019-02-28 13:50:35,713fd69b-3b57-11e9-988a-fa163e5b3bfd,NaN,NaN,VXNlcjpkZWNhNjI2Yi0zYjU1LTExZTktOTg4YS1mYTE2M2...,No,NaN,NaN,NaN,NaN,2019-02-28 13:50:35,NaN,Yes,1.0,713fd69b-3b57-11e9-988a-fa163e5b3bfd,Donner accès aux spectacles au plus grand nomb...,Développer la diffusion vidéo de spectacles un...,2019-02-28 13:50:35,NaN
1,vote,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,YES,2019-02-28 19:45:23,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,opinion,Artiste: Que seul le talent compte et non le ...,Combien d'artistes anonymes pensent percer dan...,2019-02-28 13:36:50,Yes,No,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,2019-02-28 13:36:50,Yes,No,NaN,NaN,NaN,0.0,NaN,2019-02-28 13:36:50,NaN,NaN,84fbaca0-3b55-11e9-988a-fa163e5b3bfd,VXNlcjoxYTVkNTYxOC0zM2JjLTExZTktOTg4YS1mYTE2M2...,NaN,NaN,2019-02-28 13:36:50,84fbaca0-3b55-11e9-988a-fa163e5b3bfd,NaN,NaN,VXNlcjoxYTVkNTYxOC0zM2JjLTExZTktOTg4YS1mYTE2M2...,No,NaN,NaN,NaN,NaN,2019-02-28 13:36:50,2019-02-28 13:50:35,Yes,0.0,84fbaca0-3b55-11e9-988a-fa163e5b3bfd,Artiste: Que seul le talent compte et non le ...,Combien d'artistes anonymes pensent percer dan...,2019-02-28 13:36:50,2019-02-28 13:50:35
3,opinion,Solidarité entre les fédérations,Plusieurs remarques dans le désordre:En tout p...,2019-02-28 12:55:20,Yes,No,NaN,NaN,NaN,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,2019-02-28 12:55:20,Yes,No,NaN,NaN,NaN,1.0,NaN,2019-02-28 12:55:20,NaN,NaN,b919c056-3b4f-11e9-988a-fa163e5b3bfd,VXNlcjo2ZGYwMGQ3Ni0yZWIxLTExZTktOTg4YS1mYTE2M2...,NaN,NaN,2019-02-28 12:55:20,b919c056-3b4f-11e9-988a-fa163e5b3bfd,NaN,NaN,VXNlcjo2ZGYwMGQ3Ni0yZWIxLTExZTktOTg4YS1mYTE2M2...,No,NaN,NaN,NaN,NaN,2019-02-28 12:55:20,NaN,Yes,1.0,b919c056-3b4f-11e9-988a-fa163e5b3bfd,Solidarité entre les fédérations,Plusieurs remarques dans le désordre:En tout p...,2019-02-28 12:55:20,NaN
4,vote,NaN,NaN,NaN,NaN,NaN,NaN

In [ ]:
for key in sujets_GDN:
    frames = [pd.read_csv(f"data/GDN/{date}/{sujets_GDN[key]}.csv") for date in dates_GDN]
    exp = pd.concat(frames)
    exp.to_csv(f"data/GDN/{sujets_GDN[key]}.csv", index=False)
    
    

In [ ]:
for sujet in sujets_GDN:
    df = pd.read_csv(f"data/GDN/{sujets_GDN[sujet]}.csv")
    df.drop_duplicates().to_csv(f"data/GDN/{sujets_GDN[sujet]}.csv")

In [5]:
import os

for file in os.listdir("data/VD"):
    full_path = os.path.join("data/VD", file)
    df = pd.read_csv(full_path)
    df.drop_duplicates().to_csv(full_path, index=False)

/usr/lib/python3/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (7,11,12,13,34,35,36,42,43,46,47,50,51,54,55,56,57,59,66) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/lib/python3/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,34,35,36,39,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,62,63,64,65,66) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/lib/python3/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (7,11,12,34,35,42,43,46,47,50,51,54,55,57,66) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/lib/python3/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Col

In [ ]:
s = set()

t = df[df.columns[12]].dropna()

for line in t:
    for val in line.split("|"):
        s.add(val)
    
s

In [ ]:
for col in df.columns[7:]:
    print(len(set(df[col].dropna())))
    print(set(df[col].dropna()))

In [ ]:
type_ex = [df.loc[df.type == t].iloc[0] for t in set(df["type"])]

new_df = pd.DataFrame(type_ex)

In [ ]:
new_df.to_csv("test.csv",index=False)